Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [7]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
from tqdm.auto import tqdm

In [ ]:
import numpy as np
from itertools import product

def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3: creates a weighted directed graph matrix."""
    rng = np.random.default_rng(seed)

    coords = rng.random(size=(size, 2))

    problem = rng.random((size, size))

    if negative_values:
        problem = problem * 2 - 1

    problem *= noise_level

    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            dist = np.linalg.norm(coords[a] - coords[b])
            problem[a, b] += dist
        else:
            problem[a, b] = np.inf

    np.fill_diagonal(problem, 0)

    return (problem * 1_000).round()


In [ ]:
problem = create_problem(20, density=0.15, noise_level=10, negative_values=False)

In [ ]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
has_negative = any(data['weight'] < 0 for _, _, data in G.edges(data=True))

for s, d in combinations(range(problem.shape[0]), 2):
    try:
        if has_negative:
            path = nx.bellman_ford_path(G, source=s, target=d, weight='weight')
        else:
            path = nx.dijkstra_path(G, source=s, target=d, weight='weight')

        cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    except nx.NetworkXUnbounded:
        path = None
        cost = -np.inf

    ic(s, d, cost)


# Lab 3: All-Pairs Shortest Paths
This section adds utilities to generate graphs, compute all-pairs shortest paths (using NetworkX baselines), and collect global statistics across all parameter combinations. Global stats only, no path prints.

In [ ]:

import time
import math

def build_graph(problem: np.ndarray) -> nx.DiGraph:
    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    return nx.from_numpy_array(masked, create_using=nx.DiGraph)


def all_pairs_shortest_path_lengths(G: nx.DiGraph, weight: str = "weight"):
    """Return (distances_dict, info_dict)."""
    t0 = time.perf_counter()
    negative_cycle = False
    algo = None

    # Controllo se ci sono pesi negativi
    has_negative = any(data[weight] < 0 for _, _, data in G.edges(data=True))

    try:
        if has_negative:
            algo = "bellman-ford"
            dists = dict(nx.all_pairs_bellman_ford_path_length(G, weight=weight))
        else:
            algo = "dijkstra"
            dists = dict(nx.all_pairs_dijkstra_path_length(G, weight=weight))
    except nx.NetworkXUnbounded:
        dists = {}
        negative_cycle = True

    secs = time.perf_counter() - t0
    return dists, {"algorithm": algo, "negative_cycle": negative_cycle, "seconds": secs}

def compute_global_stats(dists: dict) -> dict:
    total_pairs = 0
    reachable = 0
    pos_reachable = 0
    neg_infinite = 0
    finite_vals = []

    for s, dm in dists.items():
        for t, d in dm.items():
            if s == t:
                continue
            total_pairs += 1
            if isinstance(d, float) and math.isinf(d):
                if d < 0:
                    neg_infinite += 1
                continue
            if d == -np.inf:
                neg_infinite += 1
                continue
            if isinstance(d, float) and np.isfinite(d):
                reachable += 1
                finite_vals.append(d)
                if d > 0:
                    pos_reachable += 1

    stats = {
        "total_pairs": total_pairs,
        "reachable_pairs": reachable,
        "reachable_ratio": (reachable / total_pairs) if total_pairs else 0.0,
        "positive_pairs": pos_reachable,
        "positive_ratio": (pos_reachable / total_pairs) if total_pairs else 0.0,
        "neg_inf_pairs": neg_infinite,
    }
    if finite_vals:
        arr = np.array(finite_vals, dtype=float)
        stats.update({
            "mean": float(arr.mean()),
            "median": float(np.median(arr)),
            "min": float(arr.min()),
            "max": float(arr.max()),
        })
    else:
        stats.update({"mean": np.nan, "median": np.nan, "min": np.nan, "max": np.nan})
    return stats

In [12]:

from itertools import product


SIZES = [10, 20, 50, 100, 200, 500, 1000]
DENSITIES = [0.2, 0.5, 0.8, 1.0]
NOISE_LEVELS = [0.0, 0.1, 0.5, 0.8]
NEGATIVE_FLAGS = [False, True]



results = []

param_grid = list(product(SIZES, DENSITIES, NOISE_LEVELS, NEGATIVE_FLAGS))
print(f"Total combinations: {len(param_grid)}")

for size, density, noise_level, negative_values in tqdm(param_grid):
   
    problem = create_problem(
        size,
        density=density,
        negative_values=negative_values,
        noise_level=noise_level,
        seed=42,
    )
    G = build_graph(problem)
    dists, info = all_pairs_shortest_path_lengths(G, negative_values)
    stats = compute_global_stats(dists)

    row = {
        "size": size,
        "density": density,
        "noise_level": noise_level,
        "negative_values": negative_values,
        "algorithm": info.get("algorithm"),
        "negative_cycle": info.get("negative_cycle"),
        "seconds": info.get("seconds"),
        **stats,
    }
    results.append(row)
    print(
        f"size={size:4d} dens={density:.1f} noise={noise_level:.1f} neg={negative_values} | "
        f"algo={row['algorithm']:<12} neg_cycle={row['negative_cycle']} sec={row['seconds']:.3f} | "
        f"reach={row['reachable_ratio']:.3f} pos={row['positive_ratio']:.3f} mean={row['mean']}"
    )

print(f"Collected results: {len(results)} rows")

Total combinations: 224


  0%|          | 0/224 [00:00<?, ?it/s]

size=  10 dens=0.2 noise=0.0 neg=False | algo=dijkstra     neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=1213.5802469135801
size=  10 dens=0.2 noise=0.0 neg=True | algo=bellman-ford neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=1213.5802469135801
size=  10 dens=0.2 noise=0.1 neg=False | algo=dijkstra     neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=1314.9506172839506
size=  10 dens=0.2 noise=0.1 neg=True | algo=bellman-ford neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=1206.9382716049383
size=  10 dens=0.2 noise=0.5 neg=False | algo=dijkstra     neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=1711.2345679012346
size=  10 dens=0.2 noise=0.5 neg=True | algo=bellman-ford neg_cycle=False sec=0.000 | reach=1.000 pos=0.963 mean=1079.8024691358025
size=  10 dens=0.2 noise=0.8 neg=False | algo=dijkstra     neg_cycle=False sec=0.000 | reach=1.000 pos=1.000 mean=2004.3827160493827
size=  10 dens=0.2 noise=0.8 neg=True | algo=bellman-ford neg_cycle=Fals

In [ ]:

import csv
from datetime import datetime

if results:
    out_path = f"results_lab3_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    fieldnames = list(results[0].keys())
    with open(out_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
    print(f"Saved {len(results)} rows to {out_path}")
else:
    print("No results to save. Run the experiment loop first.")

Saved 224 rows to results_lab3_20251121_223557.csv
